In [12]:
import pandas as pd
import re



In [13]:
hotels = pd.read_csv('data/hotels.csv')

hotels.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [14]:
# Произведем поиск и удаление дубликатов (при наличии) 
hotels_dupl = hotels[hotels.duplicated()]

if hotels_dupl.shape[0] != 0:
    hotels = hotels.drop_duplicates()
    print('Имелись дубликаты в количестве {} шт. ' 
        'Данные очищены от дубликатов.'.format(hotels_dupl.shape[0]))

else:
    print('Дубликатов нет')
    
# Произведем проверку наличия пропусков 
null_cols = hotels.isnull().sum()
data_null_cols = null_cols[null_cols>0]

print(('Пропусков нет', 'Имеются пропуски, нужна обработка')
      [data_null_cols.shape[0] != 0]
      )

Имелись дубликаты в количестве 307 шт. Данные очищены от дубликатов.
Имеются пропуски, нужна обработка


In [15]:
# Кол-во строк с пропусками, вычисление процента пропусков
missed_data_qnt = hotels[hotels.isna().any(axis=1)].shape[0]
print(f'Количество строк с пропусками данных = {missed_data_qnt} \n')

missed_data_perc = (missed_data_qnt / hotels.shape[0])*100
print(f'Процент данных с пропусками = {round(missed_data_perc,2)} \n')

hotels.info()

Количество строк с пропусками данных = 2448 

Процент данных с пропусками = 0.63 

<class 'pandas.core.frame.DataFrame'>
Index: 386496 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386496 non-null  object 
 1   additional_number_of_scoring                386496 non-null  int64  
 2   review_date                                 386496 non-null  object 
 3   average_score                               386496 non-null  float64
 4   hotel_name                                  386496 non-null  object 
 5   reviewer_nationality                        386496 non-null  object 
 6   negative_review                             386496 non-null  object 
 7   review_total_negative_word_counts           386496 non-null  int64  
 8   total_number_of_reviews                     386496 non-null  int64

Из полученных данных мы видим, что пропуски имеются в части гео-данных отелей.
Проверим, возможно получится взять отсутствиующие координаты из данного датасета. Возможно, для части одних и тех же отелей они указаны, а для части нет. 

In [16]:
# Создадим список адресов отелей, коориданты которых не указаны
geo_missed_hotel_address = list(hotels[hotels['lat'].isnull()]['hotel_address'].unique())

# Пройдем циклом по DF, в поисках значений координат. Проверим по адресам отелей, 
# возможно, в каких-то строках для этих отелей будут указаны координаты

for elem in geo_missed_hotel_address:
    
    # Создадим список булевых значений по пропускам и проверим условием имеются ли
    # в нем значения False (значит координаты по отелю в DF есть).
    values_list_bool = list[
        hotels[hotels['hotel_address'] == f'{elem}']['lat'].isnull()
        ]
    
    if False in values_list_bool:
        print(f'!!! We have match for the address {elem}')
    else:
        print(f'No matches for the address {elem}')
        
# Какое количество уникальных значений пропусков 
print(f'У {len(geo_missed_hotel_address)} отелей отсутствуют гео-данные' )

No matches for the address Savoyenstra e 2 16 Ottakring 1160 Vienna Austria
No matches for the address 23 Rue Damr mont 18th arr 75018 Paris France
No matches for the address Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria
No matches for the address W hringer Stra e 33 35 09 Alsergrund 1090 Vienna Austria
No matches for the address 4 rue de la P pini re 8th arr 75008 Paris France
No matches for the address Sieveringer Stra e 4 19 D bling 1190 Vienna Austria
No matches for the address Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Austria
No matches for the address Bail n 4 6 Eixample 08010 Barcelona Spain
No matches for the address Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria
No matches for the address Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria
No matches for the address Paragonstra e 1 11 Simmering 1110 Vienna Austria
No matches for the address W hringer Stra e 12 09 Alsergrund 1090 Vienna Austria
No matches for the address 20 Rue De La Ga t 14th arr 75014 

Мы видим, что по 17 отелям отсутствуют гео-данные. 

Ввиду того, что это небольшое количество, есть возможность заполнить пропуски реальными данными, а не прибегать к методу заполнения путем применения каких-то усреденных значений. 

Для поиска координат воспользуемся обычным сервисом google-карт, который позволяет по поиску в "два клика" получить и скопировать координаты адреса. 

Создадим отдельный DataFrame, с которого потом перенесем данные в исходный DataFrame. 

In [17]:
# DF с адресами отелей с пропущенными гео-данными, и гео-денные по адресам
missed_geo_hotels_df = pd.DataFrame({
    'h_address' : ['Savoyenstra e 2 16 Ottakring 1160 Vienna Austria', 
                   '23 Rue Damr mont 18th arr 75018 Paris France', 
                   'Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria', 
                   'W hringer Stra e 33 35 09 Alsergrund 1090 Vienna Austria', 
                   '4 rue de la P pini re 8th arr 75008 Paris France',
                   'Sieveringer Stra e 4 19 D bling 1190 Vienna Austria',
                   'Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Austria',
                   'Bail n 4 6 Eixample 08010 Barcelona Spain',
                   'Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria',
                   'Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria',
                   'Paragonstra e 1 11 Simmering 1110 Vienna Austria',
                   'W hringer Stra e 12 09 Alsergrund 1090 Vienna Austria',
                   '20 Rue De La Ga t 14th arr 75014 Paris France',
                   'Hasenauerstra e 12 19 D bling 1190 Vienna Austria',
                   'Sep lveda 180 Eixample 08011 Barcelona Spain',
                   'Pau Clar s 122 Eixample 08009 Barcelona Spain',
                   'Josefst dter Stra e 22 08 Josefstadt 1080 Vienna Austria'],
    'lat' : [48.219583668024804, 48.888947389070026, 48.20962842193975, 48.22024772638908, 
        48.856838848218345, 48.24586742305909, 48.20906009960404, 41.39200796097634, 
        48.22500245912529, 48.20022540924419, 48.1638831889411, 48.224888096971725, 
        48.8400089935977, 48.24666762458812, 41.38484416401104, 41.392719066192114, 
        48.20965797171874
    ],
    'lng' : [16.2855813682633, 2.3331019682935326, 16.353311239427967, 16.355787539428473, 
             2.3047215971269517, 16.34263359781606, 16.41493483450979, 2.1756923179042045, 
             16.35692766320586, 16.391574519340875, 16.44542622468532,  16.357013493884892,  
             2.324275680907269, 16.34246193645799, 2.1609370894466107, 2.1671555526365904,
             16.351416597097717
    ]
    })

# Заполнение пропусков исходного DF данными с вновь созданного DF
hotels = hotels.set_index('hotel_address').fillna(
    missed_geo_hotels_df.set_index('h_address')).reset_index()

Этап очистки данных и заполнения пропусков окончен. Приступим к анализу признаков и работе с ними (преобразование, удаление, создание новых признаков).

In [18]:
hotels.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [19]:
# Пропишем функцию для извлечения страны из признака адреса отеля
def country_extract(string):
    country = list(string.split(' '))[-1]
    return country

# Создадим новый признак страны отеля, применив функцию к адресу
hotels['country'] = hotels['hotel_address'].apply(country_extract)

# По результату получили одно из значений 'Kingdom', 
# Напишем функцию, дающую полное название страны
hotels['country'] = hotels['country'].apply(
    lambda x: 'United Kingdom' if x == 'Kingdom' else x
    )

# Удалим признак адреса, он для модели не пригоден 
hotels = hotels.drop('hotel_address', axis=1)

hotels['country'].value_counts()
hotels
  

,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng,country
0,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671,United Kingdom
1,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097,United Kingdom
2,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643,France
3,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540,France
4,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060,Netherlands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386491,107,4/19/2017,9.0,Hotel Moonlight,France,No Negative,0,617,Tr s proche du metro Earl s court,10,10,8.8,"[' Leisure trip ', ' Group ', ' Club Double or...",106 day,51.494028,-0.191050,United Kingdom
386492,272,2/13/2017,8.4,BEST WESTERN PLUS Amedia Wien,Turkey,No Negative,0,3224,The bed was so comfy I stayed with my boyfrie...,93,1,9.2,"[' Leisure trip ', ' Couple ', ' Standard Doub...",171 day,48.192379,16.399451,Austria
386493,457,2/7/2016,6.8,Bloomsbury Palace Hotel,Netherlands,room is really small but guess is normal in L...,12,2751,great location simple check in out nice shower,9,21,8.3,"[' Business trip ', ' Solo traveler ', ' Singl...",543 day,51.520795,-0.131084,United Kingdom
386494,365,5/21/2017,8.1,The Marble Arch London,United Arab Emirates,No Negative,0,1567,Location and very comfy bed,6,28,9.2,"[' Leisure trip ', ' Solo traveler ', ' Deluxe...",74 days,51.515125,-0.160066,United Kingdom


In [20]:
stringg = 'Stratton Street Mayfair Westminster Borough London W1J 8LT United Kingdom'

cntr = list(stringg.split(' '))[-1]

cntr

'Kingdom'

In [21]:
best_hotels = pd.read_csv('data/world_top_50_hotels.csv', encoding='latin-1')
best_hotels_set = set(best_hotels['Name'].unique())


df_hotel_set = list(hotels['hotel_name'].unique())

best_hotels_set.intersection(df_hotel_set)
# The Connaught

hotels[hotels['hotel_name'] == 'The Connaught']['reviewer_score'].median()
# 9.6

hotels['reviewer_score'].median()
# 8.8

hotels['hotel_name'].value_counts()

hotel_name
Britannia International Hotel Canary Wharf           3587
Strand Palace Hotel                                  3206
Park Plaza Westminster Bridge London                 3095
Copthorne Tara Hotel London Kensington               2688
DoubleTree by Hilton Hotel London Tower of London    2379
                                                     ... 
Ibis Styles Milano Palmanova                            7
Renaissance Paris Republique Hotel Spa                  7
Hotel Wagner                                            6
Hotel Gallitzinberg                                     6
Mercure Paris Porte d Orleans                           5
Name: count, Length: 1492, dtype: int64

In [22]:
top_25 = ['Voyage Belek Golf And Spa',
 'Romance Istanbul Hotel',
 'Corpo Santo Lisbon Historical Hotel',
 'Rixos  Premium Belek - The Land Of Legends Access',
 'Hotel Moments Budapest',
 'The Blue Ivy Hotel &amp; Suites',
 'Hotel Vision Budapest',
 'Hotel Clark Budapest',
 'Ikos Aria',
 'Ikos Andalusia',
 'Quinta Jardins do Lago',
 'Ikos Oceania',
 'Hotel The Cliff Bay',
 'Palladium Boutique Hotel',
 'Hôtel Maison Mère',
 'Es Princep',
 'The Resident Covent Garden',
 'Hotel Cube',
 'The Merrion Hotel',
 'Aviator Hampshire',
 'Hotel Nordik',
 'Hôtel Grand Powers',
 'Singer Palace Hotel Roma',
 'Hôtel Bradford Elysées - Astotel',
 'Hotel Alpin Spa Tuxerhof']

top_25 = set(top_25)

hotel_name_list = list(hotels['hotel_name'].value_counts().index)

print(len(hotel_name_list))
print(len(set(hotel_name_list)))

hotel_name_set = set(hotel_name_list)

top_25.intersection(hotel_name_set)

for elem in hotel_name_list:
    if 'Season' in elem:
        print(3212)

1492
1492
3212
3212
3212


In [ ]:
import pandas as pd
import numpy as np
chain_hotels = pd.read_csv('data/chain_hotels.csv', sep=';')
chain_hotels

for i in range(chain_hotels.shape[0]):
    if chain_hotels.iloc[i][0] is np.nan:
        chain_hotels.iloc[i][0] = chain_hotels.iloc[i-1][0]

chain_hotels[chain_hotels['Managed brands'] == 'Somerset']

chain_hotels.rename(columns={'Category': 'category', 'Managed brands': 'brand'}, inplace=True)
chain_hotels['category'].unique()

# Прозивести кодировку признаков 

chain_hotels_list = list(chain_hotels['brand'])

hotel_name_list = list(hotels['hotel_name'].value_counts().index)

# for elem in chain_hotels_list:
#     elem = str(elem).lower()
#     elem = elem.split(' ')
#     elem_set = set(elem)
    
#     for elem_big in hotel_name_list:
#         elem_big = str(elem_big).lower()
#         elem_big = elem_big.split(' ')
#         elem_big_set = set(elem_big)
#         if elem_big_set.intersection(elem_set):
#             print(elem, elem_big)
            
# очистить chain_hotels от паразитов типа hotel 

def chain_hotels_cleaner(feat):
    cleaner_list = ['hotel', 'hotels' 'inn', 'garden', 'resort', 'spa', 'the', 'by']
    feat_list = str(feat).lower().split(' ')
    for word in feat_list:
        

C:\Users\nitys\AppData\Local\Temp\ipykernel_58256\4229996306.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if chain_hotels.iloc[i][0] is np.nan:
C:\Users\nitys\AppData\Local\Temp\ipykernel_58256\4229996306.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  chain_hotels.iloc[i][0] = chain_hotels.iloc[i-1][0]
C:\Users\nitys\AppData\Local\Temp\ipykernel_58256\4229996306.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pa

['raffles'] ['h', 'tel', 'le', 'royal', 'monceau', 'raffles', 'paris']
['sofitel', 'legend'] ['ink', 'hotel', 'amsterdam', 'mgallery', 'by', 'sofitel']
['sofitel', 'legend'] ['lagare', 'hotel', 'milano', 'centrale', 'mgallery', 'by', 'sofitel']
['sofitel', 'legend'] ['sofitel', 'london', 'st', 'james']
['sofitel', 'legend'] ['hotel', 'am', 'konzerthaus', 'vienna', 'mgallery', 'by', 'sofitel']
['sofitel', 'legend'] ['sofitel', 'paris', 'arc', 'de', 'triomphe']
['sofitel', 'legend'] ['sofitel', 'vienna', 'stephansdom']
['sofitel', 'legend'] ['sofitel', 'legend', 'the', 'grand', 'amsterdam']
['sofitel', 'legend'] ['h', 'tel', 'l', 'echiquier', 'op', 'ra', 'paris', 'mgallery', 'by', 'sofitel']
['sofitel', 'legend'] ['hotel', 'scribe', 'paris', 'opera', 'by', 'sofitel']
['sofitel', 'legend'] ['sofitel', 'paris', 'le', 'faubourg']
['sofitel', 'legend'] ['sofitel', 'paris', 'baltimore', 'tour', 'eiffel']
['sofitel', 'legend'] ['legend', 'saint', 'germain', 'by', 'elegancia']
['sofitel', 'lege

## Формирование множества ключевых слов, указывающих на принадлежность отеля к группе с хорошим уровнем сервиса 

Мы можем обогатить наш исходный DF данными по принадлежности отеля к определенной группе. Для этого, будут задействованы данные по чартам отелей (топ-лист), а так же, принадлежность отеля к сетевым структурам, т.к. сетевые структуры априори отличаются высоким качеством сервиса и единым подходом в сети отелей. Эти данные могут оказывать влияние на оценку, поставленную отелю. 

### Далее в работе будут использованы данные со внешних источников 
Список правообладателей представлен в файле по ссылке: 

https://docs.google.com/document/d/1PUYIL8ydxhWoZgZ_NIHCjhfQRvwLsaC01Li4m3dZ3QY/edit?usp=drive_link

### Общая стратегия следующая: 

Задействованы три источника данных: 

1. Скаченный датасет с kaggle.com, представляющий топ-50 отелей в мире. 
2. Парсинга сайта tripadviser.com, конкретно страницы с топ-25 отелей Европы. 
3. Копирование данных с сайта wikipedia.com и перенос их в формат CSV. 

Далее, по каждому блоку будет произведено преообразование в части вычленения названий отелей и их разбивка на отдельные слова и формирование списков слов по каждому блоку. После список преобразуется в множество для удаления повторяющихся значений.

Затем, множества по трем блокам объединяются в одно множество. 

Очистка множества от общеупотребимых в части отелей слов как the, inn, hotel, resort и прочее. 

#### Топ-50 отелей в мире

Датасет был перегружен из источника на гугл-диск. Представленные в коде ссылки видут на перезалитый файл. 

In [3]:
import pandas as pd

url='https://drive.google.com/file/d/1ATDWtM5RP-OtiFLUuePr5VGzcP3Cpdtx/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
top_50_world_df = pd.read_csv(url, encoding='latin-1')

# top_50_world_df = pd.read_csv('data\world_top_50_hotels.csv', encoding='latin-1')

top_50_world_df

,Rank,Name,Location,Overview,Total Rooms,Starting Rate in ($),Dining Area,Drinking Area,Hotel Ammenties,Address,Number
0,1,Capella Bangkok,Bangkok,A calming riverside oasis in the centre of cac...,101,600,Côte by Mauro Colagreco and Phra Nakhon,Stella,"Swimming pool(s), spa/wellness centre, fitnes...","300, 2 Charoen Krung Rd, Yan Nawa, Sathon, Ban...",+66 2 098 3888
1,2,Passalacqua,Lake Como,A sumptuous Lake Como retreat made for enjoyin...,24,1300,The Garden Terrace,Bar Terrace,"Spa/wellness centre, fitness centre, family f...","Via Besana, 59, 22010 Moltrasio CO, Italy",+39 031 44311
2,3,Rosewood Hong Kong,Hong Kong,Masterful minimalism meets marble-lined luxury,413,650,Bluhouse and The Dining Room,Darkside,"Swimming pool, spa/wellness centre, fitness c...","18 Salisbury Rd, Tsim Sha Tsui, Hong Kong",+852 3891 8888
3,4,Cheval Blanc,Paris,Uber-stylish riverside luxury and world-class ...,72,3000,Langosteria,Le Tout-Paris,"Swimming pool(s), spa/wellness centre, fitnes...","8 Quai du Louvre, 75001 Paris, France",+33 1 40 28 00 00
4,5,The Upper House,Hong Kong,A temple of Zen high above Hong Kong,117,650,Salisterra,NaN,"Swimming pool, fitness centre, family friendly","88 Queensway, Admiralty, Hong Kong",NaN
5,6,Raffles Singapore,Singapore,History marries hospitality at this original c...,115,2000,Butcher's Block,Long Bar,"Swimming pool(s), spa/wellness centre, fitnes...","1 Beach Rd, Singapore 189673",+65 6337 1886
6,7,Aman Tokyo,Tokyo,"Light-filled, spacious escape in bustling Tokyo",84,1800,Arva Restaurant,The Lounge by Aman,"Swimming pool(s), spa/wellness centre, fitnes...","The Otemachi Tower, 1 Chome-5-6 ?temachi, Chiy...",+81 3-5224-3333
7,8,Soneva Fushi,Maldives,Sublime island luxury with a focus on sustaina...,72,2800,"Once Upon a Table and So Hands On by Akira, a...",By the Bar,"Beach, swimming pool(s), spa/wellness centre,...","Kunfunadhoo Island, Eydhafushi, Maldives",+960 660-0304
8,9,Atlantis The Royal,Dubai,The epitome of over-the-top extravagance,795,460,Dinner by Heston Blumenthal and Jaleo by José...,Cloud 22,"Over 90 swimming pools, Skyblaze fountain, wa...","The Palm Jumeirah, Crescent Rd, Dubai, United ...",+971 4 426 3000
9,10,Nihi Sumba,Sumba Island,"Refined, rustic luxury in remote Indonesia",27,1300,Ombak,Nio Beach Club & Pool,"Beach, chocolate factory, equestrian stables,...","Hoba Wawi, Wanokaka, West Sumba Regency, East ...",+62 361 757149


In [ ]:
# pd.set_option('display.max_rows', None)

chain_hotels

def brand_filling(brand):
    brand = str(brand).lower().split(' ')
    for elem in 
    
    

# Надо просто свести бренды к одному списку и проверять на вхождение названий отелей в этот список. Очень сложно сделать разбивку на кетегории, т.к. имеются бренды с разными категориями 

,category,brand
0,Luxury,Raffles
1,Luxury,Orient Express
2,Luxury,Delano
3,Luxury,SO/
4,Luxury,Sofitel Legend
5,Luxury,Fairmont
6,Luxury,SLS
7,Luxury,Sofitel
8,Luxury,The House of Originals
9,Luxury,Rixos


In [44]:
a = {1, 5, 3}
b = {2, 5, 6}
if a.intersection(b):
    print('yes')
else:
    print('no')

yes
